# Step 1: Merging & Cleaning for Project 5

In [115]:
# Imports:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

import warnings

warnings.filterwarnings('ignore')

In [2]:
# Import the primary provider datasets

providers= pd.read_csv("../../data/NH_ProviderInfo_Nov2021.csv")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
print(providers.shape)

(15264, 88)


In [4]:
# Import the 2020 covid dataset

covid20= pd.read_csv("../../data/faclevel_2020.csv")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
print(covid20.shape)

(491177, 228)


In [5]:
# Import the 2021 covid dataset

covid21= pd.read_csv("../../data/faclevel_2021.csv")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
print(covid21.shape)

(748867, 228)


In [68]:
# Import the vaccination data from Sam 

vaccinations= pd.read_csv("../../data/cleaned_vax_pcts.csv")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
print(vaccinations.shape)

(15264, 3)


In [76]:
# Import the income data from Shirley

income= pd.read_csv("../../data/cleaned_income2018.csv")
income["Federal Provider Number"]= income["Federal Provider Number"].astype(str)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
print(income.shape)

(15137, 120)


In [77]:
income.dtypes

Unnamed: 0                                          int64
rpt_rec_num                                         int64
Federal Provider Number                            object
Facility Name                                      object
Street Address                                     object
                                                   ...   
Total Income                                      float64
Net Income                                        float64
Inpatient PPS Amount                              float64
Nursing and Allied Health Education Activities    float64
Allowable Bad Debts                               float64
Length: 120, dtype: object

In [47]:
providers.dtypes

Federal Provider Number              object
Provider Name                        object
Provider Address                     object
Provider City                        object
Provider State                       object
                                     ...   
Total Amount of Fines in Dollars    float64
Number of Payment Denials             int64
Total Number of Penalties             int64
Location                             object
Processing Date                      object
Length: 88, dtype: object

In [45]:
covid20.dtypes

Week Ending                                                                                                                                                  object
Federal Provider Number                                                                                                                                      object
Provider Name                                                                                                                                                object
Provider Address                                                                                                                                             object
Provider City                                                                                                                                                object
                                                                                                                                                             ...   
Percentage of Cu

In [44]:
covid21.dtypes

Week Ending                                                                                                                                                  object
Federal Provider Number                                                                                                                                      object
Provider Name                                                                                                                                                object
Provider Address                                                                                                                                             object
Provider City                                                                                                                                                object
                                                                                                                                                             ...   
Percentage of Cu

In [64]:
#Sanity check of number of unique facilities
covid20['Federal Provider Number'].nunique()

15460

In [6]:
#retain last record of 2020 to get total yearly cases/deaths per 1000 residents
last_2020_rec=covid20[covid20['Week Ending']=='12/27/2020']

In [17]:
#retain latest record of 2021 to get total yearly cases/deaths per 1000 residents
last_2021_rec=covid21[covid21['Week Ending']=='12/05/21']

In [36]:
#filter to needed fields for 2020 target data 
target_2020=last_2020_rec[['Federal Provider Number','Total Resident Confirmed COVID-19 Cases Per 1,000 Residents', 'Total Resident COVID-19 Deaths Per 1,000 Residents']]

In [55]:
#rename the columns so we can differentiate between years
new_columns_dict_2020 = {
    'Total Resident Confirmed COVID-19 Cases Per 1,000 Residents': '2020_cases_per_1000',
    'Total Resident COVID-19 Deaths Per 1,000 Residents': '2020_deaths_per_1000',
}
target_2020.rename(columns=new_columns_dict_2020, inplace=True)
target_2020["Federal Provider Number"]= target_2020["Federal Provider Number"].astype(str)
target_2020.head()

,Federal Provider Number,2020_cases_per_1000,2020_deaths_per_1000
31,15009,905.66,94.34
63,15010,295.08,32.79
95,15012,392.86,107.14
127,15014,465.75,0.00
159,15015,139.53,0.00


In [38]:
#filter to needed fields for 2021 target data 
target_2021=last_2021_rec[['Federal Provider Number','Total Resident Confirmed COVID-19 Cases Per 1,000 Residents', 'Total Resident COVID-19 Deaths Per 1,000 Residents']]

In [56]:
new_columns_dict_2021 = {
    'Total Resident Confirmed COVID-19 Cases Per 1,000 Residents': '2021_cases_per_1000',
    'Total Resident COVID-19 Deaths Per 1,000 Residents': '2021_deaths_per_1000',
}
target_2021["Federal Provider Number"]= target_2021["Federal Provider Number"].astype(str)
target_2021.rename(columns=new_columns_dict_2021, inplace=True)
target_2021.head()

,Federal Provider Number,2021_cases_per_1000,2021_deaths_per_1000
48,15009,905.66,94.34
97,15010,276.32,26.32
146,15012,305.56,83.33
195,15014,871.43,28.57
244,15015,324.68,0.00


In [57]:
providers["Federal Provider Number"]= providers["Federal Provider Number"].astype(str)

In [33]:
target_2020.isnull().sum()

Federal Provider Number      0
2020_cases_per_1000        271
2020_cases_per_1000        271
dtype: int64

In [34]:
target_2021.isnull().sum()

Federal Provider Number      0
2021_cases_per_1000        393
2021_cases_per_1000        393
dtype: int64

In [19]:
#Sanity check of number of unique facilities
Target_2020['Federal Provider Number'].nunique()

15331

In [20]:
#Sanity check of number of unique facilities
Target_2021['Federal Provider Number'].nunique()

15249

In [21]:
#Sanity check of number of unique facilities
providers['Federal Provider Number'].nunique()

15264

## Merge Datasets

In [58]:
# Add in 2020 targets, merge to providers dataset 

merge1=pd.merge(providers,target_2020, how='left', on=["Federal Provider Number"])
merge1.isnull().sum()

Federal Provider Number        0
Provider Name                  0
Provider Address               0
Provider City                  0
Provider State                 0
                            ... 
Total Number of Penalties      0
Location                       0
Processing Date                0
2020_cases_per_1000          676
2020_deaths_per_1000         676
Length: 90, dtype: int64

In [59]:
merge1['Federal Provider Number'].nunique()

15264

In [62]:
# Add in 2021 targets, merge to previous dataset 

merge2=pd.merge(merge1, target_2021, how='left', on=["Federal Provider Number"])
merge2.isnull().sum()

Federal Provider Number       0
Provider Name                 0
Provider Address              0
Provider City                 0
Provider State                0
                           ... 
Processing Date               0
2020_cases_per_1000         676
2020_deaths_per_1000        676
2021_cases_per_1000        1334
2021_deaths_per_1000       1334
Length: 92, dtype: int64

In [63]:
merge2['Federal Provider Number'].nunique()

15264

In [72]:
# Add in vaccinations data, merge to previous dataset 
merge3=pd.merge(merge2, vaccinations, how='left', on=["Federal Provider Number"])
merge3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15264 entries, 0 to 15263
Data columns (total 94 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Federal Provider Number                                          15264 non-null  object 
 1   Provider Name                                                    15264 non-null  object 
 2   Provider Address                                                 15264 non-null  object 
 3   Provider City                                                    15264 non-null  object 
 4   Provider State                                                   15264 non-null  object 
 5   Provider Zip Code                                                15264 non-null  int64  
 6   Provider Phone Number                                            15264 non-null  int64  
 7   Provider SSA County Code                

In [70]:
#sanity check that no providers were dropped 
merge3['Federal Provider Number'].nunique()

15264

In [91]:
# Add in income data, merge to previous dataset 
merge4=pd.merge(merge3, income, how='left', on=["Federal Provider Number"])
merge4['Federal Provider Number'].nunique()

15264

In [92]:
merge4.head()

,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,Ownership Type,Number of Certified Beds,Average Number of Residents per Day,Average Number of Residents per Day Footnote,Provider Type,Provider Resides in Hospital,Legal Business Name,Date First Approved to Provide Medicare and Medicaid Services,Continuing Care Retirement Community,Special Focus Status,Abuse Icon,Most Recent Health Inspection More Than 2 Years Ago,Provider Changed Ownership in Last 12 Months,With a Resident and Family Council,Automatic Sprinkler Systems in All Required Areas,Overall Rating,Overall Rating Footnote,Health Inspection Rating,Health Inspection Rating Footnote,QM Rating,QM Rating Footnote,Long-Stay QM Rating,Long-Stay QM Rating Footnote,Short-Stay QM Rating,Short-Stay QM Rating Footnote,Staffing Rating,Staffing Rating Footnote,RN Staffing Rating,RN Staffing Rating Footnote,Reported Staffing Footnote,Physical Therapist Staffing Footnote,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,Reported RN Staffing Hours per Resident per Day,Reported Licensed Staffing Hours per Resident per Day,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Case-Mix Nurse Aide Staffing Hours per Resident per Day,Case-Mix LPN Staffing Hours per Resident per Day,Case-Mix RN Staffing Hours per Resident per Day,Case-Mix Total Nurse Staffing Hours per Resident per Day,Adjusted Nurse Aide Staffing Hours per Resident per Day,Adjusted LPN Staffing Hours per Resident per Day,Adjusted RN Staffing Hours per Resident per Day,Adjusted Total Nurse Staffing Hours per Resident per Day,Rating Cycle 1 Standard Survey Health Date,Rating Cycle 1 Total Number of Health Deficiencies,Rating Cycle 1 Number of Standard Health Deficiencies,Rating Cycle 1 Number of Complaint Health Deficiencies,Rating Cycle 1 Health Deficiency Score,Rating Cycle 1 Number of Health Revisits,Rating Cycle 1 Health Revisit Score,Rating Cycle 1 Total Health Score,Rating Cycle 2 Standard Health Survey Date,Rating Cycle 2 Total Number of Health Deficiencies,Rating Cycle 2 Number of Standard Health Deficiencies,Rating Cycle 2 Number of Complaint Health Deficiencies,Rating Cycle 2 Health Deficiency Score,Rating Cycle 2 Number of Health Revisits,Rating Cycle 2 Health Revisit Score,Rating Cycle 2 Total Health Score,Rating Cycle 3 Standard Health Survey Date,Rating Cycle 3 Total Number of Health Deficiencies,Rating Cycle 3 Number of Standard Health Deficiencies,Rating Cycle 3 Number of Complaint Health Deficiencies,Rating Cycle 3 Health Deficiency Score,Rating Cycle 3 Number of Health Revisits,Rating Cycle 3 Health Revisit Score,Rating Cycle 3 Total Health Score,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Processing Date,2020_cases_per_1000,2020_deaths_per_1000,2021_cases_per_1000,2021_deaths_per_1000,Percent Vaccinated Residents,Percent Vaccinated Healthcare Personnel,Unnamed: 0,rpt_rec_num,Facility Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural versus Urban,Fiscal Year Begin Date,Fiscal Year End Date,Type of Control,Total Days Title V,Total Days Title XVIII,Total Days Title XIX,Total Days Other,Total Days Total,Number of Beds,Total Bed Days Available,Total Discharges Title V,Total Discharges Title XVIII,Total Discharges Title XIX,Total Discharges Title Other,Total Discharges Total,SNF Average Length of Stay Title V,SNF Average Length of Stay Title XVIII,SNF Average Length of Stay Title XIX,SNF Average Length of Stay Total,SNF Admissions Title V,SNF Admissions Title XVIII,SNF Admissions Title XIX,SNF Admissions Other,SNF Admissio

In [94]:
merge5['Federal Provider Number'].nunique()

15264

## Remove records where covid cases/deaths are missing

In [99]:
merge4clean1 = merge4[merge4['2020_cases_per_1000'].notna()]

In [101]:
merge4clean2 = merge4clean1[merge4clean1['2021_cases_per_1000'].notna()]

In [103]:
merge4clean3 = merge4clean2[merge4clean2['2020_deaths_per_1000'].notna()]

In [106]:
merge4clean4 = merge4clean3[merge4clean3['2021_deaths_per_1000'].notna()]

In [108]:
merge4clean4['Federal Provider Number'].nunique()

13718

In [113]:
merge4clean4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14260 entries, 0 to 15842
Columns: 213 entries, Federal Provider Number to Allowable Bad Debts
dtypes: float64(151), int64(9), object(53)
memory usage: 23.3+ MB


## Write Final Dataset

In [114]:
#write final datset to CSV
merge4clean4.to_csv("../../data/NH_merged.csv")